In [ ]:
from PIL import Image
import os, glob
import numpy as np
from sklearn import model_selection

classes = ["dog", "cat", "monkey"]
num_classes = len(classes)
image_size = 150

# 画像の読み込み
X = []
Y = []
for index, classlabel in enumerate(classes):
    photos_dir = "./" + classlabel
    files = glob.glob(photos_dir + "/*.jpg")
    for i, file in enumerate(files):
        if i >= 200: break
        image = Image.open(file)
        image = image.convert("RGB")
        image = image.resize((image_size, image_size))
        data = np.asarray(image)
        X.append(data)
        Y.append(index)

X = np.array(X)
Y = np.array(Y)

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, Y)
xy = (X_train, X_test, y_train, y_test)
np.save("./imagefiles_apli.npy", xy)

In [ ]:
from keras.models import Sequential,Model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import np_utils
import keras
import numpy as np
from keras.applications import VGG16
from keras.callbacks import EarlyStopping,ModelCheckpoint


classes = ["dog", "cat", "monkey"]
num_classes = len(classes)
image_size = 150

# データの読み込み
X_train, X_test, y_train, y_test = np.load("./imagefiles_apli.npy",allow_pickle=True)
#データ生成時に実行済み
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)
X_train = X_train.astype("float") / 256
X_test = X_test.astype("float") / 256

model = VGG16(weights='imagenet',include_top=False, input_shape=X_train.shape[1:])

top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(num_classes, activation='softmax'))

model = Model(inputs = model.input, outputs=top_model(model.output))
model.summary()

for layer in model.layers[:15]:
    layer.trainable = False

callbacks = [EarlyStopping(monitor="val_loss", patience=2),
                             ModelCheckpoint(filepath = "dcm_CNN.h5",monitor = "val_loss", save_best_only = True)]
opt = keras.optimizers.adam(lr=0.0001, decay=1e-6)
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=32, callbacks = callbacks, epochs=30, validation_split=0.3)

# モデルの保存
model.save('./dcm_CNN.h5')

scores = model.evaluate(X_test, y_test, verbose=1)
print('Test Loss: ', scores[0])
print('Test Accuracy: ', scores[1])